# OpenStreetMap Udacity Project

## Map area

### Reno, Nevada

https://www.openstreetmap.org/relation/170120

I chose this area because I recently moved to Nevada and I thought it would be interesting to do some research on a city close to me. There is so much rural area around me that this seemed like a safe bet for finding interesting information worthy of wrangling! It is worth noting that this project tested me. I felt I had a reasonable, novice-level, grasp of programming in C++. I question everything now. I struggled through this and had to refer to stackoverflow.com quite often for what feels like basic things. I believe I have accomplished what the goal of the project is, to show a general understanding of wrangling and perform general queries on said wrangled data. I hope this project also conveys this!

## This is the sample code provided by Udacity which was used for sample.osm creation.

Some minor changes were done to account for a bytes/str error I was recieving.

### sample.py

In [6]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "reno.osm"  # Replace this with your osm file
SAMPLE_FILE = "sample.osm"

k = 10 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n') # ADDED "b" TO ACCOUNT FOR BYTES/STRING ERROR *****************
    output.write(b'<osm>\n  ') # ADDED "b" TO ACCOUNT FOR BYTES/STRING ERROR ***************************************************

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write(b'</osm>') # ADDED "b" TO ACCOUNT FOR BYTES/STRING ERROR *********************************************************

## This is code provided by Udacity with the necessary updates for the audit procedure.

While analyzing the output of the sample.osm, some problems encountered were found in addresses, zip codes, and city names. Addresses were not consistent with the use of several abbreviations, zip codes may have white space characters and extra info beyond five numbers, and Reno was inconsistently listed as a city. The audit.py file provided by Udacity was updated to audit these issues. Though there are many more potential issues, for this project we will focus on general data cleaning for query purposes. We will focus on cleaning street names, zip codes, and adding Reno to all city fields as data was only for the Reno area. I left a long trailing "*" (asterisk) to indicate changes I made in the files provided.

### audit.py

In [16]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "reno.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#UPDATED EXPECTED AS WELL *************************************************************************************************************
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Loop", "Way", "View", "Circle", "Row", "CreekTrail", "40",
            "Terrace"]

# UPDATE THIS VARIABLE, UPDATED *****************************************************************************************************
mapping = { "St": "Street",
            "St.": "Street",
            "st": "Street",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "blvd:": "Boulevard",
            "Pkwy": "Parkway",
            "pkwy": "Parkway",
            "Ln": "Lane",
            "pl": "Place",
            "Ct": "Court",
            "Pl": "Place",
            "Dr": "Drive",
            "Pl.": "Place",
            "Dr": "Drive"
            }


def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
    return street_types #FIXED


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(OSMFILE):
    osm_file = open(OSMFILE, "r")
    street_types = defaultdict(set)
    postal_code_types = defaultdict(set)
    
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
                if is_postal_code(tag):
                    audit_postal_code(postal_code_types, tag.attrib['v'])

    osm_file.close()
    return street_types, postal_code_types


# YOUR CODE STARTS **********************************************************************************************************************************************************************************************************


# CHARS FOR POSTAL CODE
def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")


# DICTIONARY FOR POSTAL CODES MODELED FROM audit_steet_type
def audit_postal_code(postal_code_types, postal_code):  
    if not postal_code.isupper() or ' ' not in postal_code:
        postal_code_types['case_whitespace_problems'].add(postal_code)
    else:
        postal_code_types['other'].add(postal_code)
    return postal_code_types


# POSTAL CODE TEST MODELED FROM test
def postal_code_test(): 
    postcode_types = audit(OSMFILE)[1]
    pprint.pprint(dict(postcode_types))

    for postcode_type, postcodes in postcode_types.items():
        for postcode in postcodes:
            better_postcode = update_postal_code(postcode)
            print(postcode, "=>", better_postcode)
            

# YOUR CODE ENDS **********************************************************************************************************************************************************************************************************
  

def test():
    st_types = audit(OSMFILE)[0]#ADDED FOR DICT, FIXED
    len(st_types) == 5
    pprint.pprint(dict(st_types))

    for st_type, ways in st_types.items():
        for name in ways:
            better_name = update_name(name, mapping)
            print(name, "=>", better_name)
            if name == "West Lexington St.":
                better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                better_name == "Baldwin Road"
                

#if __name__ == '__main__':
#    test()
#    postal_code_test()


Some of this is adopted from audit.py, such as "expected" and "mapping". These helper functions will be imported for use in data.py. I also found it helpful to isolate code that worked while troubleshooting my code and it just became easier to import this file for this purpose.

## fix.py

In [17]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSMFILE = "reno.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

#UPDATED EXPECTED AS WELL
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Loop", "Way", "View", "Circle", "Row", "CreekTrail", "40",
            "Terrace"]

# UPDATE THIS VARIABLE, UPDATED
mapping = { "St": "Street",
            "St.": "Street",
            "st": "Street",
            "Ave": "Avenue",
            "Blvd": "Boulevard",
            "blvd:": "Boulevard",
            "Pkwy": "Parkway",
            "pkwy": "Parkway",
            "Ln": "Lane",
            "pl": "Place",
            "Ct": "Court",
            "Pl": "Place",
            "Dr": "Drive",
            "Pl.": "Place",
            "Dr": "Drive"
            }

# DICTIONARY FOR POSTAL CODES MODELED FROM audit_steet_type
def audit_postal_code(postal_code_types, postal_code):  
    if not postal_code.isupper() or ' ' not in postal_code:
        postal_code_types['case_whitespace_problems'].add(postal_code)
    else:
        postal_code_types['other'].add(postal_code)
    return postal_code_types

# REMOVE WHITESPACE OR ANY EXTRA CHARS BEYOND 5
def update_postal_code(postal_code):
    postal_code = postal_code.upper()
    if ' ' not in postal_code:
        if len(postal_code) != 5:
            postal_code = postal_code[0:5]
    return postal_code # changed from - postal_code

# UPDATE STREET NAMES
mapping_keys = []
for k,v in mapping.items():
    mapping_keys.append(k)

def update_names(name, mapping):
    m = street_type_re.search(name)
    if name == 'sparks place':
        return name.title()
    elif name == 'Sparks Square - 3':
        return 'Sparks Square'
    elif m:
        bad_suffix = m.group()
        if m.group() in mapping_keys: #l
            good_suffix = mapping[bad_suffix]
            return re.sub(bad_suffix,good_suffix,name)
        else:
            return name
    else: 
        return name

# ADD RENO AS CITY IF NOT THERE OR MISSPELLED
def add_reno():
    osm_file = open(OSMFILE, "r")  
    city_list = set()
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if tag.attrib['k'] == "addr:city" and tag.attrib['v'] != "Reno":
                    city_list.add(tag.attrib['v'])
    return city_list

if __name__ == "__main__":
    add_reno()


## This is code provided by Udacity with the necessary updates for the .osm to .csv  procedure.

Utilized the code from data.py, as given by Udacity, to parse .osm into .csv files. Have included fix.py as an import. fix.py contains functions for cleaning streets, zip codes, and city names. Encountered a problem with the UnicodeDictWriter function which required updating to be compatible with Python 3. Used the update provided by WGU. I left a long trailing "*" (asterisk) to indicate changes I made in the files provided.


### data.py

In [18]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

import fix #IMPORT fix.py FOR CLEANING FUNCTIONS AND WILL ADD RENO TO ANY MISSED CITY FIELD

OSM_PATH = "reno.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

#HELPER FUNCTIONS FROM fix.py FOR CLEANING STREET AND POSTAL CODE ***************************************************************************
def update_tags(tag):
    if tag['key'] == "street":
        tag['value'] = fix.update_name(tag['value'], fix.mapping)

    elif tag['key'] == "postcode":
        tag['value'] = fix.update_postal_code(tag['value'])



def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

# YOUR CODE STARTS HERE **********************************************************************************************************************************************************************************************************
# PARSE AND CREATE .csv's CALL UPDATE_TAGS WITHIN FIX.PY FOR CLEANING FUNCTION, THIS WORKS!!!! 5:30

    if element.tag == 'node':
        for field in NODE_FIELDS: 
            v_field = element.attrib[field]
            node_attribs[field] = v_field
            
        for child in element: 
            tag = {}
            vk = child.attrib['k']
            tag['id'] = node_attribs['id']
            
            if PROBLEMCHARS.search(vk): 
                continue
                
            elif LOWER_COLON.search(vk): 
                vk_split = vk.split(':', 1)
                tag['type'] = vk_split[0]
                tag['key'] = vk_split[1]
                tag['value'] = child.attrib['v']
            
            else: 
                tag['key'] = vk
                tag['value'] = child.attrib['v']
                tag['type'] = default_tag_type

            update_tags(tag) # FIX.PY 
            tags.append(tag) 
        
        return {'node': node_attribs, 'node_tags': tags}
        
    elif element.tag == 'way':

        for field in WAY_FIELDS: 
            v_field = element.attrib[field]
            way_attribs[field] = v_field
        
        counter = 0
        for child in element:
            tag = {}
            if child.tag == 'tag':  
                vk = child.attrib['k']
                tag['id'] = way_attribs['id']
                
                if PROBLEMCHARS.search(vk): 
                    continue
                    
                elif LOWER_COLON.search(vk): 
                    vk_split = vk.split(':', 1)
                    tag['type'] = vk_split[0]
                    tag['key'] = vk_split[1]
                    tag['value'] = child.attrib['v']
                
                else: 
                    tag['key'] = vk
                    tag['value'] = child.attrib['v']
                    tag['type'] = default_tag_type

                update_tags(tag) # FIX.PY
                tags.append(tag) 
            
            if child.tag =='nd': 
                way_node = {}
                way_node['id'] = way_attribs['id']
                way_node['node_id'] = child.attrib['ref']
                way_node['position'] = counter
                counter += 1
                way_nodes.append(way_node)
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
# WORKS, FINALLY 5:30 ******************************************************************************************************************

# ================================================== #
#               Helper Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )

# FIXED UNICODEDICTWRITER ERROR PER INFO FROM WGU CHATTER, PYTHON 2 ERRORS *******************************************************************************
        
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: v for k, v in row.items()
        })
    

# ================================================== #
#               Main Function                        #
# ================================================== #

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""
#ADDED UTF FOR OPEN *******************************************************************************************************
    with codecs.open(NODES_PATH, 'w', "utf-8") as nodes_file, \
            codecs.open(NODE_TAGS_PATH, 'w', "utf-8") as nodes_tags_file, \
            codecs.open(WAYS_PATH, 'w', "utf-8") as ways_file, \
            codecs.open(WAY_NODES_PATH, 'w', "utf-8") as way_nodes_file, \
            codecs.open(WAY_TAGS_PATH, 'w', "utf-8") as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    fix.add_reno()
    process_map(OSM_PATH, validate=True)

## Created the .db file and read .csv to .db using python.

I tried using one of my more favorite platforms, TablePlus, to create the database file but ran into quite a few problems specific to that program and decided to resort to python. This was a bit of a challenge with several bytes and string errors along the way but once I got one table to work right, the rest were easy. 

In [34]:
import csv
import sqlite3

con = sqlite3.connect("reno.db")
con.text_factory = str
cur = con.cursor()

# CREATE TABLE NODES
cur.execute("create table nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','r') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("insert into nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                values (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

# CREATE TABLE NODES_TAGS
cur.execute("create table nodes_tags (id, key, value, type);")
with open('nodes_tags.csv','r') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("insert into nodes_tags (id, key, value, type) values (?, ?, ?, ?);", to_db)
con.commit()

# CREATE TABLE WAYS
cur.execute("create table ways (id, user, uid, version, changeset, timestamp);")
with open('ways.csv','r') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("insert into ways (id, user, uid, version, changeset, timestamp) values (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

# CREATE TABLE WAYS_NODES
cur.execute("create table ways_nodes (id, node_id, position);")
with open('ways_nodes.csv','r') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("insert into ways_nodes (id, node_id, position) values (?, ?, ?);", to_db)
con.commit()

# CREATE TABLE WAYS_TAGS
cur.execute("create table ways_tags (id, key, value, type);")
with open('ways_tags.csv','r') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("insert into ways_tags (id, key, value, type) values (?, ?, ?, ?);", to_db)
con.commit()

# Overview of the data

In [35]:
import os

osm_path = 'reno.osm'
db_path = 'reno.db'
ways = 'ways.csv'
ways_tags = 'ways_tags.csv'
ways_nodes = 'ways_nodes.csv'
nodes = 'nodes.csv'
nodes_tags = 'nodes_tags.csv'
print('Files for database...')
print('reno.osm :', int(round(os.stat(osm_path).st_size) / (1024.0 * 1024.0)), 'MB')
print('reno.db :', int(round(os.stat(db_path).st_size) / (1024.0 * 1024.0)), 'MB')
print('ways.csv :', int(round(os.stat(ways).st_size) / (1024.0 * 1024.0)), 'MB')
print('ways_tags.csv :', int(round(os.stat(ways_tags).st_size) / (1024.0 * 1024.0)), 'MB')
print('ways_nodes.csv :', int(round(os.stat(ways_nodes).st_size) / (1024.0 * 1024.0)), 'MB')
print('nodes.csv :', int(round(os.stat(nodes).st_size) / (1024.0 * 1024.0)), 'MB')
print('nodes_tags.csv :', int(round(os.stat(nodes_tags).st_size) / (1024.0 * 1024.0)), 'MB')

Files for database...
reno.osm : 222 MB
reno.db : 153 MB
ways.csv : 4 MB
ways_tags.csv : 9 MB
ways_nodes.csv : 29 MB
nodes.csv : 89 MB
nodes_tags.csv : 1 MB


### Loading sql and sql database than performing querys.

In this section I load sqlite3 and conduct several queries for various info. I broke my queries out individually for a better viewing experience.

In [36]:
#SETUP FOR .DB CONNECTION AND QUERYS

import sys

import sqlite3
from pprint import pprint
import pandas as pd

database_file = 'reno.db'
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

In [37]:
def user_count():
    uc = pd.read_sql_query("select count(sub.uid) \
                            from (select uid \
                            from nodes union \
                            select uid \
                            from ways) sub", conn)
    pprint("Total number of users contributing...")
    pprint(uc)
user_count()

'Total number of users contributing...'
   count(sub.uid)
0             828


In [38]:
def nodes_count():
    nc = pd.read_sql_query("select count(*) from nodes", conn)
    pprint("Total number of nodes...")
    pprint(nc)
nodes_count()

'Total number of nodes...'
   count(*)
0   1107582


In [39]:
def ways_count():
    wc = pd.read_sql_query("select count(*) from ways", conn)
    pprint("Total number of ways...")
    pprint(wc)
ways_count()

'Total number of ways...'
   count(*)
0     77659


In [40]:
def amenities_count():
    ac = pd.read_sql_query("select value, count(*) as num \
                            from nodes_tags where key = 'amenity' \
                            group by value \
                            order by num desc \
                            limit 15;", conn)
    pprint("Total number of amenities within sample.osm...")
    pprint(ac)
amenities_count()

'Total number of amenities within sample.osm...'
               value  num
0         restaurant  121
1              bench  110
2             school   71
3          fast_food   61
4               fuel   38
5                bar   31
6   parking_entrance   30
7               cafe   30
8   charging_station   22
9            toilets   19
10              bank   19
11          post_box   18
12           shelter   16
13               pub   12
14          pharmacy   12


In [41]:
def postal_code_counts():
    pcc = pd.read_sql_query("select subq.value, count(*) as count \
                             from (select * from nodes_tags union \
                             select * from ways_tags) \
                             subq where subq.key = 'postcode' \
                             group by subq.value \
                             order by count desc \
                             limit 15", conn)
    pprint("Top 15 postal codes and the counts within...")
    pprint(pcc)
postal_code_counts()

'Top 15 postal codes and the counts within...'
    value  count
0   89509   2031
1   89503    708
2   89519    598
3   89501    382
4   89512    290
5   89506    245
6   89502    150
7   89436    106
8   89521     78
9   89523     54
10  89431     44
11  89511     35
12  89434     29
13  89439     17
14  96161     14


## Other ideas about the datasets

This represents a few other queries to see some other data viewpoints. These viewpoints represent more opportunities to clean our data. For example; in amenities count, we could potentially combine pub and bar but this may misrepresent the data as they could represent different entities, after all, in Reno, Nevada alcohol and liquor are literally sold everywhere and many casinos also have bars. Another example of opportunity rest in building type; there are not 132 physical universities in Reno, as a city in a valley, it just isn't big enough to have that many independent universities but we do have the University of Northern Reno (UNR) which could easily have 132 structures that could fall under the university umbrella. Many opportunities exist with this data set depending on the motives of the user.

In [42]:
def get_places():
    gp = pd.read_sql_query("select sub.value, nt.value \
                            from nodes_tags nt join \
                                (select id, value \
                                from nodes_tags \
                                where key = 'place') \
                            sub on nt.id = sub.id \
                            where nt.key = 'name' \
                            limit 10", conn)
    pprint("Places within sample taken from reno.osm...")
    pprint(gp)
get_places()

'Places within sample taken from reno.osm...'
               value                     value
0           locality                    Hinton
1      neighbourhood        Sierra Subdivision
2  isolated_dwelling                   Iceland
3             hamlet                   Peavine
4      neighbourhood  Prosser Lakeview Estates
5           locality              Hobart Mills
6             hamlet                Hirschdale
7           locality                    Mystic
8           locality                      Boca
9             hamlet                    Plumas


In [43]:
def building_type():
    bt = pd.read_sql_query("select tags.value, count(*) as num \
                            from (select * from nodes_tags union all select * from ways_tags) tags \
                            where tags.key = 'building' and tags.value != 'yes' \
                            group by tags.value \
                            order by num desc \
                            limit 15;", conn)
    pprint("Top 5 buildings by type...")
    pprint(bt)
building_type()

'Top 5 buildings by type...'
         value   num
0        house  5327
1   apartments   535
2   commercial   382
3       retail   271
4         roof   243
5       garage   178
6   industrial   153
7   university   132
8         shed    91
9       school    80
10     terrace    62
11   warehouse    46
12     garages    37
13      office    28
14      hangar    27


In [44]:
def sports_types():
    st = pd.read_sql_query("select tags.value, count(*) as num \
                            from (select * from nodes_tags union all \
                                select * from ways_tags) tags \
                            where tags.key = 'sport' and tags.value != 'yes' \
                            group by tags.value \
                            order by num desc \
                            limit 10;", conn)
    pprint("Top 5 sports land types...")
    pprint(st)
sports_types()

'Top 5 sports land types...'
             value  num
0           tennis   98
1         baseball   89
2             golf   78
3       basketball   64
4       horseshoes   34
5          karting   21
6  beachvolleyball   15
7           soccer   14
8         swimming   10
9       skateboard    9


In [45]:
def web_site():
    ws = pd.read_sql_query("select distinct tags.value \
                            from (select * from nodes_tags \
                                union all \
                                select * from ways_tags) tags \
                            where tags.key = 'website' \
                            order by tags.value \
                            limit 10;", conn)
    pprint("Some website's...")
    pprint(ws)
web_site()

"Some website's..."
                                               value
0                           facebook.com/redrockreno
1                                     greatbasin.org
2                                http://adsd.nv.gov/
3                          http://coneyislandbar.net
4  http://corporate.ppg.com/Our-Company/Worldwide...
5             http://deserthighlandsministorage.com/
6                 http://discountwindowcoverings.org
7  http://dpbh.nv.gov/About/Overview/NNAMHS_Overv...
8                            http://drmccaskill.com/
9                          http://golfwildcreek.com/


In [46]:
def top_contrib():
    tc = pd.read_sql_query("select e.user, count(*) as num \
                            from (select user from nodes union all \
                                  select user from ways) e \
                            group by e.user \
                            order by num desc \
                            limit 10;", conn)
    pprint("Top 10 contributors...")
    pprint(tc)
top_contrib()

'Top 10 contributors...'
              user     num
0        btwhite92  507179
1  woodpeck_fixbot  108440
2         abschiff   63731
3          nmixter   61840
4         balcoath   57741
5   BryanOSullivan   29792
6         Phil O'D   25304
7            mgwst   14956
8          JPSReno   11367
9     MatthewKluft   11359


In [47]:
def popular_eats():
    pe = pd.read_sql_query("select nodes_tags.value, count(*) as num \
                            from nodes_tags \
                                join (select distinct(id) \
                                from nodes_tags \
                                where value='restaurant') i \
                                on nodes_tags.id=i.id \
                            where nodes_tags.key='cuisine' \
                            group by nodes_tags.value \
                            order by num desc \
                            limit 10;", conn)
    pprint("Eateries within sample.osm...")
    pprint(pe)
popular_eats()

'Eateries within sample.osm...'
         value  num
0        pizza   16
1      chinese   10
2      mexican    9
3     japanese    6
4      italian    5
5       indian    4
6     american    4
7   vietnamese    3
8     sandwich    3
9  coffee_shop    3


# Conclusion

The sheer amount of data that makes up the various files was very eye-opening. I completely understand how a vast majority of a data analyst's time can be given to just trying to get the data clean. With that said, I believe this data is clean enough for the purpose of the class but when considering things like the websites, Wikipedia pages, and phone numbers; there is ample opportunity for much more wrangling. 